In [1]:
import os
import os.path as osp
import sys
import torch
import numpy as np
import scipy.sparse as sp
from torch.utils.data import DataLoader
from torch_geometric.nn import Node2Vec
from torch_geometric.utils import from_scipy_sparse_matrix
from torch_geometric.data import Data
from scipy import io
from tqdm import tqdm
import json
import shutil

In [2]:
PARAMS = {
"GRAPH_NAME": "graph_1.mat",
"BATCH_SIZE": 128,
"EMBEDDING_DIM": 128,
"WALK_LENGTH": 100,
"CONTEXT_SIZE": 10,
"WALKS_PER_NODE": 10,
"LEARNING_RATE": 1e-2,
"NUM_EPOCH": 5,
"CUDA":True,
"P":1,
"Q":1,
"EMBEDDING_NAME": "graph_1.npy"
}

In [3]:
fp = '/datasets/dsc180a-wi20-public/Malware/group_data/group_02/sensitive_data'

In [4]:
N = io.loadmat(osp.join(fp, 'interim', 'graph', PARAMS['GRAPH_NAME']))['N']

In [10]:
io.loadmat(osp.join(fp, 'interim', 'graph', PARAMS['GRAPH_NAME']))

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Tue May 26 04:24:00 2020',
 '__version__': '1.0',
 '__globals__': [],
 'N': <585320x585320 sparse matrix of type '<class 'numpy.int64'>'
 	with 2604806 stored elements in Compressed Sparse Column format>,
 'U': <481907x481907 sparse matrix of type '<class 'numpy.int64'>'
 	with 805055 stored elements in Compressed Sparse Column format>,
 'A': <481907x103413 sparse matrix of type '<class 'numpy.int64'>'
 	with 103413 stored elements in Compressed Sparse Column format>,
 'P': <103413x481907 sparse matrix of type '<class 'numpy.int64'>'
 	with 1696338 stored elements in Compressed Sparse Column format>,
 'post_label': array([[0, 0, 0, ..., 0, 0, 0]]),
 'post_cate': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'post_indx': a

In [5]:
edge_idx, x =from_scipy_sparse_matrix(N)
post_indx = io.loadmat(osp.join(fp, 'interim', 'graph', PARAMS['GRAPH_NAME']))['post_indx']
post_indx = torch.from_numpy(post_indx.reshape(-1,))
data = Data(
    x = x,
    edge_index = edge_idx
)

In [6]:
loader = DataLoader(post_indx, batch_size=PARAMS['BATCH_SIZE'], shuffle=False)
if PARAMS['CUDA']:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
else:
    device = 'cpu'

In [9]:
model = Node2Vec(data.edge_index, embedding_dim=PARAMS['EMBEDDING_DIM'], walk_length=PARAMS['WALK_LENGTH'],
                    context_size=PARAMS['CONTEXT_SIZE'], walks_per_node=PARAMS['WALKS_PER_NODE'], p = PARAMS['P'], q = PARAMS['Q'], sparse=True)

In [13]:
model(post_indx)

tensor([[ 0.0108, -0.3249, -0.6775,  ..., -0.3880,  1.1678,  1.0462],
        [-0.4706, -0.3753, -0.2350,  ...,  0.0106,  0.0690, -0.3072],
        [-0.9684,  1.5481,  1.0705,  ..., -0.2417,  1.5498, -0.5845],
        ...,
        [ 0.7267, -0.0541, -1.0990,  ..., -0.1199, -0.0054,  0.2474],
        [ 0.4417,  0.2297,  0.8620,  ...,  0.5328, -0.6913,  0.2474],
        [-1.5310,  0.1646, -2.6391,  ..., -0.3694, -0.2230,  0.6893]],
       grad_fn=<IndexBackward>)